<a href="https://colab.research.google.com/github/MichaelSeitz98/Seminararbeit/blob/main/KindDetektion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Google Drive mounten für Zugriff auf Datensatz und zum Speichern der trainierten Modelle



In [2]:
# Verknüpfung mit meiner Google Drive herstellen

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Unbedingt vor jedem Trainingslauf ANPASSEN! SONST GANZES TRAININNG ÜBERSCHRIEBEN!

training_folder = '/content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100'

%mkdir {training_folder}
# Anpassen!

mkdir: cannot create directory ‘/content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100’: File exists


In [4]:
# Sicherstellen, dass train_labels.csv dort vorhanden sind (in keinem Unterornder)

datensatz_folder = '/content/drive/MyDrive/DS2_POOLG_80_20_20'

##Vor jedem neuen Trainingslauf bearbeiten!

Alle für Training benötigten Dateien werden in diesen Ordner auf der Drive abgespeichert.
 1.) train.record
 2.) test.record 
 3.) test_labels.csv
 4.) train_labels.csv
 5.) .config File
 6.) labelMap (optional) 
 7.) dann noch das generateTF_Record_Script  



## Installation der Tensorflow OD API

Muss bei jeder neuen Laufzeit neu installiert werden - auch für die Inferenzanwendung.

In [5]:
# Allgemeines Tensorflow installieren 
!pip install tensorflow=="2.6.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564.4 MB 2.4 kB/s 
     |████████████████████████████████| 132 kB 15.1 MB/s 
     |████████████████████████████████| 14.8 MB 119 kB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=f1760ebed61a896994ed1357a09b3368266e6757f6b2c68203830fc8101b8350
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=135d73f6cc756f196f5116e62e0c193609b003c02d35040f745bfa3d15ab6025
  Stored in directory: /root/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68706 sha256=23ad88c8bdf74cf0d1eebba6867635556c24e50c15c7e4d76adcf2461e5edbb1
  Stored in directory: /root/.cache/pip/wheels/62/7

In [6]:
# Tensorflow Modelle laden (wenn noch nicht vorhanden)
import pathlib
import os
 
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3529, done.
remote: Counting objects: 100% (3529/3529), done.
remote: Compressing objects: 100% (2942/2942), done.
remote: Total 3529 (delta 931), reused 1524 (delta 533), pack-reused 0
Receiving objects: 100% (3529/3529), 47.00 MiB | 28.87 MiB/s, done.
Resolving deltas: 100% (931/931), done.


In [9]:
!python -m pip --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [7]:
# Installation der Tensorflow Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696454 sha256=bff860b1c9549370d3c51cc13c52858d94809f666a96ba53cc9647bcedda51e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-a67xd729/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78545 sha256=2092c4bea820a82b1416389971661d580d00565eeaa8a3fb424c723b51900f2f
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=f093d2ccf9454433abc1de62531914e0d4c0f88ef224be045b4578de0bd58503
  Stored in directory: /root/.cache/pip/wheels/d6/e5/b1/6b151d9b535ee50aaa6ab27d145a0104b6df02e5636f0376da
  Created wheel for docopt: filename=docopt-

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [8]:
# Test, ob initiales Set-up erfolgreich

!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2022-11-12 13:54:34.928135: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 13:54:35.088158: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-12 13:54:35.117942: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-12 13:54:35.932412: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

## Datenvorbereitung für Training 



In [10]:
# In den Traingsornder springen
%cd {training_folder}

/content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100


In [12]:
# Generate TF-Record-Skript von meinem GitHub-Account laden, alternativ einfach Generate TF-Record in die Drive hochladen

!wget https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/44688230a2a4692e06b2ab927ab7abca0eb163ec/generate_tfrecord.py

--2022-11-12 13:11:07--  https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/44688230a2a4692e06b2ab927ab7abca0eb163ec/generate_tfrecord.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3394 (3.3K) [text/plain]
Saving to: ‘generate_tfrecord.py’

generate_tfrecord.p 100%[===================>]   3.31K  --.-KB/s    in 0.02s   

2022-11-12 13:11:07 (169 KB/s) - ‘generate_tfrecord.py’ saved [3394/3394]



In [13]:
# LabelMap von meinem GitHub-Accountin den Trainingsordner laden

!wget https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/main/labelmap.pbtxt

--2022-11-12 13:11:07--  https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/main/labelmap.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37 [text/plain]
Saving to: ‘labelmap.pbtxt’

labelmap.pbtxt      100%[===================>]      37  --.-KB/s    in 0s      

2022-11-12 13:11:08 (681 KB/s) - ‘labelmap.pbtxt’ saved [37/37]



In [11]:
# train_labels.csv und validatation_labels.csv offline erzeugen
# Wichtig: Den Datensatzpfad anpassen in der xml_to_csv offline-Datei

print(f'Datensatz Pfad: {datensatz_folder}')

train_labels_path = datensatz_folder + '/train_labels.csv'
validation_labels_path = datensatz_folder + '/validation_labels.csv'

!cp {train_labels_path} {training_folder}
!cp {validation_labels_path} {training_folder}

print (f'Training labels: {train_labels_path}')
print (f'Validation labels: {validation_labels_path}')

Datensatz Pfad: /content/drive/MyDrive/DS2_POOLG_80_20_20
Training labels: /content/drive/MyDrive/DS2_POOLG_80_20_20/train_labels.csv
Validation labels: /content/drive/MyDrive/DS2_POOLG_80_20_20/validation_labels.csv


In [54]:
# Erstellung der TF-Record Dateien
# Kann bis zu 5 Minuten dauern

print(f'{datensatz_folder}/train')
!python generate_tfrecord.py --csv_input={train_labels_path} --image_dir={datensatz_folder}/train --output_path=train.record
!python generate_tfrecord.py --csv_input={validation_labels_path} --image_dir={datensatz_folder}/validation --output_path=validation.record

/content/drive/MyDrive/DS2_POOLG_80_20_20/train
2022-11-12 13:34:17.376446: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 13:34:17.623373: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-12 13:34:18.551280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-12 13:34:18.551413: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic lib

In [15]:
# Dateipfade aller fürs Training relevanten Dateien: train.record, test.record, labelmap.pbtxt, + dann noch: config des Modelles, Alle aus dem Trainings-Ordner ziehen

train_record_path = training_folder + '/train.record'
validation_record_path =  training_folder + '/validation.record'
labelmap_path =  training_folder + '/labelmap.pbtxt'

print (f'Diese Datein müssen vorhanden sein in: {training_folder}\n')
print (f'train_record_path: {train_record_path} \nvalidation_record_path: {validation_record_path}\nlabelmap_path: {labelmap_path}')

Diese Datein müssen vorhanden sein in: /content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100

train_record_path: /content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100/train.record 
validation_record_path: /content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100/validation.record
labelmap_path: /content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100/labelmap.pbtxt


## Modellauswahl und Trainingsparameter festlegen...

Modellauswahl aus: (https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).

In [13]:
# Hier können die entscheidenede Trainingsparamter festgelegt werde. Variablen für jedes Training neu setzen/überprüfen. 
# Modellnamen raussuchen von:  https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

name_model = 'efficientdet_d1_coco17_tpu-32'
name_model_als_tar_gz = 'efficientdet_d1_coco17_tpu-32.tar.gz'

batch_size = 8
num_steps = 20000
num_eval_steps = 100
nr_of_classes = 1

In [32]:
# Vortrainiertes Modell aus TF Model Zoo laden. An dieser Stelle Modelle austauschen! 
#!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
#!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
#!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz


download_link_model = f'http://download.tensorflow.org/models/object_detection/tf2/20200711/{name_model_als_tar_gz}'
print(download_link_model)
!wget {download_link_model}

print (f'Download des Modells: {download_link_model}' )

http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
--2022-11-12 13:18:44--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 209.85.234.128, 2607:f8b0:4001:c17::80
Connecting to download.tensorflow.org (download.tensorflow.org)|209.85.234.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51839363 (49M) [application/x-tar]
Saving to: ‘efficientdet_d1_coco17_tpu-32.tar.gz’

efficientdet_d1_coc 100%[===================>]  49.44M  79.4MB/s    in 0.6s    

2022-11-12 13:18:45 (79.4 MB/s) - ‘efficientdet_d1_coco17_tpu-32.tar.gz’ saved [51839363/51839363]

Download des Modells: http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz


In [33]:
# Das Modell unzippen. Es wird ein neuer Ordner im Trainings-Ordner erstellt 

!tar xf {name_model_als_tar_gz}
print (f'Entpacke Datei: {name_model_als_tar_gz}')

Entpacke Datei: efficientdet_d1_coco17_tpu-32.tar.gz


In [34]:
# Lösche MODEL-NAME.tar.gz, da nicht mehr benötigt

!rm {name_model_als_tar_gz}

In [36]:
# Hier wird der Ausgangspunkt für das Training gesetzt. Ab diesem Punkt wird mit den eigenen Daten traininert (Konzept des Transfer-Learnings)

fine_tune_checkpoint = f'{training_folder}/{name_model}/checkpoint/ckpt-0'

print(fine_tune_checkpoint)

/content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100/efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0


In [ ]:
# Jetzt das Config-File für Training runterladen aus ModelZoo Configs 
# Alternativ einfach das pipeline.config aus /content/efficientdet_d0_coco17_tpu-32/pipeline.config verwenden (sind leicht unterschiedlich) 

#!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config
#!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config


#!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/{name_model}.config
# !wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d1_640x640_coco17_tpu-8.config

# und als Config_path setzen: 

#
#config_path = f'{name_model}.config'    # bei allem außer EfficientDet funktionierend

#print (f'Config path gesetzt auf: {config_path}')

In [14]:
config_path = f'{training_folder}/{name_model}/pipeline.config'
print (f'Config Path gesetzt auf: {config_path}')

Config Path gesetzt auf: /content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100/efficientdet_d1_coco17_tpu-32/pipeline.config


In [50]:
# config file bearbeiten mit den eigenen Daten
# Anzahl der Klassen anpassen

import re

with open(config_path) as f:
    config = f.read()

with open(config_path, 'w') as f:
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  #  fine_tune_checkpoint Pfad setzen
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(validation_record_path), config)
  
  # Anzahl der Detektionsklassen setzen
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(nr_of_classes), config)
  
  # batch size 
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  #nur bei Train die batch_size ändern. Bei eval auf 1 lassen
  
  # training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # fine-tune checkpoint & type 'detection'
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [51]:
%pwd

'/content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100'

In [ ]:
# Anzeigen, der eben geänderten Datei

print(config_path)
%cat {config_path}


## Ab hier: alles vorhanden für Training. Training starten

In [15]:
print (config_path)

/content/drive/MyDrive/training2_effdetd1_ds2_8_20000_100/efficientdet_d1_coco17_tpu-32/pipeline.config


In [ ]:
#Starten eines Trainingsvorgangs.

# mit regulären Metrics

!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={config_path} \
    --model_dir={training_folder} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps} \
    --checkpoint_every_n=1000


2022-11-12 13:56:47.072338: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 13:56:47.234679: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-12 13:56:47.264420: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-12 13:56:48.111468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [17]:
# Erzeugt final Evaluation metrics z.B. mAP / iou precisions -> auch unabhängig von Training anwendbar

!python /content/models/research/object_detection/model_main_tf2.py --model_dir={training_folder} --pipeline_config_path={config_path} --checkpoint_dir={training_folder}

2022-11-12 16:44:08.633154: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 16:44:08.790052: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-12 16:44:08.819658: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-12 16:44:09.640204: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

## Export model inference graph

In [18]:
output_directory = training_folder+'/inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {training_folder} \
    --output_directory {output_directory} \
    --pipeline_config_path {config_path}

2022-11-12 16:45:12.969377: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 16:45:13.120608: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-12 16:45:13.149495: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-12 16:45:13.972539: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

## Anwendung des trainierten Model

Dieser Schritt im Inferenz-Notebook unabhängig vom Training ausführbar. 

In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

In [ ]:
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'{output_directory}/saved_model')

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
#model_dir = '/content/drive/MyDrive/training_effDet_d0_4_3000_100/'
#print (model_dir)
#speicherPfadEvalBilder = model_dir + "angewendeteBilder"

#%mkdir {speicherPfadEvalBilder}
print(training_folder)

In [ ]:
#model_dir = '/content/drive/MyDrive/training_effDet_d0_4_3000_100/'
print (training_folder)
speicherPfadEvalBilder = training_folder + "/angewendeteBilder"

%mkdir {speicherPfadEvalBilder}

for image_path in glob.glob('/content/drive/MyDrive/Child_Dataset/testEval/*g'):

  image_name = os.path.basename(image_path).replace(".jpg", "")
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(model, image_np)
  outImg = vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      min_score_thresh=0.5,
      max_boxes_to_draw=20,
      line_thickness=3)
  
  im = Image.fromarray(outImg)
  display(Image.fromarray(image_np))
  print ('Versuche hier zu speichern:' + speicherPfadEvalBilder + "/" + image_name + "_boxed.jpg")
  im.save(speicherPfadEvalBilder + "/" + image_name + "_boxed.jpg")
